# Vectorizing Raw Data: N-Grams

### N-Grams 

Creates a document-term matrix where counts still occupy the cell but instead of the columns representing single terms, they represent all combinations of adjacent words of length n in your text.

"NLP is an interesting topic"

| n | Name      | Tokens                                                         |
|---|-----------|----------------------------------------------------------------|
| 2 | bigram    | ["nlp is", "is an", "an interesting", "interesting topic"]    |
| 3 | trigram   | ["nlp is an", "is an interesting", "an interesting topic"]  |
| 4 | four-gram | ["nlp is an interesting", "is an interesting topic"]  |

### Read in text

In [4]:
import pandas as pd
import re
import string
import nltk
pd.set_option('display.max_colwidth', 100)

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()
wn = nltk.WordNetLemmatizer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

### Create function to remove punctuation, tokenize, remove stopwords, and stem

In [5]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = " ".join([wn.lemmatize(word) for word in tokens if word not in stopwords])
    return text

data['cleaned_text'] = data['body_text'].apply(lambda x: clean_text(x))
data.head()

,label,body_text,cleaned_text
0,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s
1,ham,"Nah I don't think he goes to usf, he lives around here though",nah dont think go usf life around though
2,ham,Even my brother is not like to speak with me. They treat me like aids patent.,even brother like speak treat like aid patent
3,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,date sunday
4,ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,per request melle melle oru minnaminunginte nurungu vettam set callertune caller press 9 copy friend callertune


### Apply CountVectorizer (w/ N-Grams)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

ngram_vect = CountVectorizer(ngram_range=(2,2))
X_counts = ngram_vect.fit_transform(data['cleaned_text'])
print(X_counts.shape)
print(ngram_vect.get_feature_names_out())

(5567, 31621)
['008704050406 sp' '0089my last' '0121 2025050' ... 'üll submitting'
 'üll take' '〨ud evening']


### Apply CountVectorizer (w/ N-Grams) to smaller sample

In [11]:
data_sample = data[0:20]

In [13]:
ngram_vect_sample = CountVectorizer(ngram_range=(2,2))
X_counts_sample = ngram_vect_sample.fit_transform(data_sample['cleaned_text'])
print(X_counts_sample.shape)
print(ngram_vect_sample.get_feature_names_out())

(20, 198)
['09061701461 claim' '100 20000' '100000 prize' '11 month' '12 hour'
 '150pday 6days' '16 tsandcs' '20000 pound' '2005 text' '21st may'
 '4txtú120 poboxox36504w45wq' '6days 16' '81010 tc' '87077 eg'
 '87077 trywales' '87121 receive' '87575 cost' '900 prize' 'aft finish'
 'aid patent' 'alright way' 'anymore tonight' 'apply 08452810075over18s'
 'apply reply' 'ard smth' 'around though' 'brother like'
 'call 09061701461' 'call mobile' 'caller press' 'callertune caller'
 'camera free' 'cash 100' 'chance win' 'claim 81010' 'claim call'
 'claim code' 'click httpwap' 'click wap' 'co free' 'code kl341'
 'colour mobile' 'comp win' 'copy friend' 'cost 150pday' 'credit click'
 'cried enough' 'csh11 send' 'cup final' 'customer selected' 'da stock'
 'date sunday' 'dont miss' 'dont think' 'dont want' 'eg england'
 'eh remember' 'england 87077' 'england macedonia' 'enough today'
 'entitled update' 'entry questionstd' 'entry wkly' 'even brother'
 'fa 87121' 'fa cup' 'feel way' 'ffffffffff alr

### Vectorizers output sparse matrices

_**Sparse Matrix**: A matrix in which most entries are 0. In the interest of efficient storage, a sparse matrix will be stored by only storing the locations of the non-zero elements._

In [16]:
X_counts_df = pd.DataFrame(X_counts_sample.toarray())
X_counts_df.columns = ngram_vect_sample.get_feature_names_out()
X_counts_df

,09061701461 claim,100 20000,100000 prize,11 month,12 hour,150pday 6days,16 tsandcs,20000 pound,2005 text,21st may,4txtú120 poboxox36504w45wq,6days 16,81010 tc,87077 eg,87077 trywales,87121 receive,87575 cost,900 prize,aft finish,aid patent,alright way,anymore tonight,apply 08452810075over18s,apply reply,ard smth,around though,brother like,call 09061701461,call mobile,caller press,callertune caller,camera free,cash 100,chance win,claim 81010,claim call,claim code,click httpwap,click wap,co free,code kl341,colour mobile,comp win,copy friend,cost 150pday,credit click,cried enough,csh11 send,cup final,customer selected,da stock,date sunday,dont miss,dont think,dont want,eg england,eh remember,england 87077,england macedonia,enough today,entitled update,entry questionstd,entry wkly,even brother,fa 87121,fa cup,feel way,ffffffffff alright,final tkts,fine way,finish lunch,finish ur,first lar,free 08002986030,free call,free entry,free membership,friend callertune,go str,go usf,goalsteam news,going try,gonna home,ha ha,ha joking,hl info,home soon,httpwap xxxmobilemovieclubcomnqjkgighjjgcbl,im gonna,ive cried,jackpot txt,kim watching,kl341 valid,lar da,latest colour,lccltd pobox,life around,like aid,like speak,link next,lor ard,lor finish,lunch already,lunch go,macedonia dont,make wet,may 2005,meet sooner,melle melle,melle oru,membership 100000,message click,minnaminunginte nurungu,miss goalsteam,mobile 11,mobile camera,mobile update,month entitled,month ha,nah dont,name yes,national team,naughty make,network customer,news txt,next txt,nurungu vettam,oh kim,oru minnaminunginte,pay first,per request,pobox 4403ldnw1a7rw18,poboxox36504w45wq 16,pound txt,press copy,prize jackpot,prize reward,questionstd txt,ratetcs apply,receive entry,receivea 900,remember spell,reply hl,request melle,reward claim,scotland 4txtú120,selected receivea,send 87575,seriously spell,set callertune,six chance,smth lor,soon dont,speak treat,spell name,stock comin,str lor,stuff anymore,talk stuff,tc wwwdbuknet,team 87077,text fa,think go,tkts 21st,tonight ive,treat like,try month,trywales scotland,tsandcs apply,txt csh11,txt message,txt ratetcs,txt ur,txt word,update co,update latest,ur lunch,ur national,urgent week,use credit,usf life,valid 12,valued network,vettam set,want talk,wap link,way feel,way gota,way meet,week free,win cash,win fa,winner valued,wkly comp,word claim,wwwdbuknet lccltd,xxxmobilemovieclub use,yes naughty
0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0